In [59]:
import numpy as np

In [60]:
import pandas as pd

In [61]:
import tensorflow as tf
%tensorflow_version 2.x

In [62]:
text = open('harry_potter.txt',mode='r',encoding='latin1').read()

In [63]:
print(text[:200])

Doorstep Delivery
[It is nighttime in Surrey, we see two owls on the street sign "PRIVET DRIVE" and the camera pans to the street with very identical looking brown bricked houses. One the owls fly awa


In [64]:
len(text)

418058

In [65]:
vocab = sorted(set(text))

In [66]:
len(vocab)

88

In [67]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [68]:
char_to_ind[':']

21

In [69]:
ind_to_char = np.array(vocab)

In [70]:
ind_to_char[21]

':'

In [71]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [72]:
encoded_text[:200]

array([27, 66, 66, 69, 70, 71, 56, 67,  1, 27, 56, 63, 60, 73, 56, 69, 76,
        0, 50, 32, 71,  1, 60, 70,  1, 65, 60, 58, 59, 71, 71, 60, 64, 56,
        1, 60, 65,  1, 42, 72, 69, 69, 56, 76,  7,  1, 74, 56,  1, 70, 56,
       56,  1, 71, 74, 66,  1, 66, 74, 63, 70,  1, 66, 65,  1, 71, 59, 56,
        1, 70, 71, 69, 56, 56, 71,  1, 70, 60, 58, 65,  1,  3, 39, 41, 32,
       45, 28, 43,  1, 27, 41, 32, 45, 28,  3,  1, 52, 65, 55,  1, 71, 59,
       56,  1, 54, 52, 64, 56, 69, 52,  1, 67, 52, 65, 70,  1, 71, 66,  1,
       71, 59, 56,  1, 70, 71, 69, 56, 56, 71,  1, 74, 60, 71, 59,  1, 73,
       56, 69, 76,  1, 60, 55, 56, 65, 71, 60, 54, 52, 63,  1, 63, 66, 66,
       62, 60, 65, 58,  1, 53, 69, 66, 74, 65,  1, 53, 69, 60, 54, 62, 56,
       55,  1, 59, 66, 72, 70, 56, 70,  9,  1, 38, 65, 56,  1, 71, 59, 56,
        1, 66, 74, 63, 70,  1, 57, 63, 76,  1, 52, 74, 52])

In [73]:
encoded_text.shape

(418058,)

In [74]:
print(text[200000:200500])

 the dementors seem to have developed an interest in you... perhaps I should teach you.

But after the holidays.

For now, I need to rest.


Last call for Hogsmeade!

Come on, now!

Guys, let me go.

Clever, Harry.

But not clever enough.

We've got a better way.

I'm trying to get to Hogsmeade.

We know.

We'll get you there.

We'll show you a quicker way.

Lf you pipe down.

Bless him.

Let me go! Come on, guys. Don't...

Now, Harry.

Come and join the big boys.

What are you doing?!

What's t


In [75]:
lines = """
Last call for Hogsmeade!

Come on, now!

Guys, let me go.

Clever, Harry.

But not clever enough.

We've got a better way.

"""

In [76]:
len(lines)

125

In [77]:
seq_len = 130

In [78]:
total_num_len = len(text)//(seq_len+1)

In [79]:
total_num_len

3191

In [80]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [81]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [82]:
def create_seq_targets(seq):
  input_txt = seq[:-1]
  target_txt = seq[1:]
  return input_txt, target_txt

In [83]:
dataset = sequences.map(create_seq_targets)

In [84]:
batch_size = 128
buffer_size = 10000

In [85]:
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

In [86]:
vocab_size = len(vocab)
embed_dim = 68
rnn_neurons=1026

In [87]:
from tensorflow.keras.models import Sequential

In [88]:
from tensorflow.keras.layers import Dense, Embedding, GRU

In [89]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [90]:
def sparse_cat_loss(y_true, y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [91]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
  model = Sequential()
  model.add(Embedding(vocab_size, embed_dim, batch_input_shape = [batch_size, None]))
  model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  model.compile(optimizer = 'adam', loss = sparse_cat_loss)
  return model

In [92]:
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [93]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (128, None, 68)           5984      
_________________________________________________________________
gru_4 (GRU)                  (128, None, 1026)         3373488   
_________________________________________________________________
dense_4 (Dense)              (128, None, 88)           90376     
Total params: 3,469,848
Trainable params: 3,469,848
Non-trainable params: 0
_________________________________________________________________


In [94]:
#epochs = 60
#model.fit(dataset, epochs=epochs)

In [95]:
#model.save('harry_potter_updated.h5')

In [96]:
from tensorflow.keras.models import load_model

In [97]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('harry_potter_updated.h5')
model.build(tf.TensorShape([1, None]))

In [98]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [102]:
print(generate_text(model,"Harry Potter",gen_size=500))

Harry Potter?
- Who it's losting
it. Harry said as Flitwick gasps, a vorcers are door again] Petrified with wizards. He looks past from sevening whistle looks at him, he trips, accompanied by his assem tair it again. Hagrid closes the can; just a dry. The sky cheers at his most prince got past a bubble knouging. The hell monithres at Harry with one of Liffle Poldrraxis is noingey... misturn into a student why you wanted him at the skyoun with an interesting as it wasnat
in the locket,
but nothing muck hallo


In [104]:
print(generate_text(model,"Harry",gen_size=500))

Harry never apsifed that what.

RON
We need to know.

HARRY
I didn't amsomet de intry-plouders! Pleakers, fools when I say if I might has been changed into Forget to his kneese, snakeshing the zooks fayis evening. It's Lovel use your referess, snawes, and camely friends are being and into the mirror. I never enough to get into the light]
And there's somelgist behind them. They walk once as be descined at she gives a second blood saflie cocker and practice.
Harry: [realizing] The Pirith will be the si


In [103]:
print(generate_text(model," Ron Weasley",gen_size=500))

 Ron Weasley ..."
"...... I could to say that be
waken in there. Students raughter dragons
arenting not just any only jumps.

RON: I must wondered it.
-hud appiarm
Wize it.
Could I he, you looking for him..

Did and Fluck from his begroom coming.
Draco: Oh. Every didn't take it
of two platse against the
Ronsten Harry's last one.
Harry: Sorry about what Snape and thut
castle, kill! [Harry sits screaming and runs through the castle. Lever mendione they can't

All right.

There's no need. It's never any hister


In [106]:
print(generate_text(model,"Ron",gen_size=500))

RonY
On never gever
Ruddis has as bad a smyle.
- What's ghisp---NYE-have hards.

HARRY: I brigge has never told me how to deserve it.
- Come on, get back, this is of Harry Potter?
Harry: But you mean, I had notide achid about the dust brother. A clum wills up a boot and stinking]
Frinte? Greg thats his Camelloom steps their wands] Toking you will lose. We sound of

Harry Potter?
Parchis tean fate friends.

RON
Grey lade closes
? just... Then, two, two. Three-Itho open it. Snape turns Mad, Professor


In [107]:
print(generate_text(model,"Hermione Granger",gen_size=500))

Hermione Granger, of Dobonet.

It's been indentione Goile?
Harry: I'm a couple moster, hav it, ilvisible
ofared fe.

Now, who's dead.

Sure?

Leter me, Sirius.
- Thank you, Mr. Filch...

That is my pruce.

I've gone more.

Fiest to put it three, you join
me intreduct I think I had a give a wishy's o
junce doesen.
I thought stide, you have done.



EXT. HOGWARTS

I can two shape one erspun and in.

HERMIONE
Harry!

HARRY
Clearly, in my opens, normoll, and we soing uncreams.

And then Dumbledore, giving us all. B


In [108]:
print(generate_text(model,"Hermione",gen_size=500))

Hermione You-Know-Who?

Not, I will do something!

Ancond, the blud than familier this evening.

I live with your name is staffi...... is it. [falls into the still of intr classes at once!

Hermione east,

Harry: Wingardium Leviosa! [His is ition, it was mentaps, and I'll be killed.

I'm sure I saw you something more
Varno.

HARRY
The curse it's a sirey side in hy ask, the sinks my children desestedding the statue. the car waiseed out and evilants the can'le family following him after him.
Snape: Well, 


In [109]:
print(generate_text(model,"Draco Malfoy",gen_size=500))

Draco Malfoy get up.
- Where did you know...?
Dumbledore as Just witch on wizard if you
chose of you think that I am our seek this Sorting
Hat: Ahe him]
Ginny. No, That's thy In short. [Harry shouts. Harry callly be sous
boying with the students were.
Couspo-ition they are not.
- Lugging.
I thought I could have staircase to es.
Harry and Ron stare mys, falling flies off. Here strugg...
...these way
be just the snake.

STRICHER 1NSPEAVE! The Purilish is smoke.


INT. ROOMT
How did you detic, me am
Iffounder 


In [110]:
print(generate_text(model,"Draco",gen_size=500))

Dracon gets out of it.] Ev. And Mimbst more. They kill you...
...you were waiting for you.

GADY
Ah, gon.

Ifter he should think I can end their new
Smyous. These was with the broom! Why, imagine this boys. They start are stoom?

HARRY
Yeah she does.

Sur an annoran and their driver granity
an ermeds.

WERRIONE
I think, Mad-Eye fould readized there but it's coming up!

ate me gets to be with, Harry's brilliant on the wrong nors]
Professor Dumbledore: Ginny, Cornelius Fudge is a poriticialy

the secon


In [111]:
print(generate_text(model,"Albus Dumbledore",gen_size=500))

Albus Dumbledore trailly, Mr. Foldow me with you...knaw-MY-
HERMIONE
Exactly.

and spells in standing op the wall.
Magical Chouts and starts turn] Mins Granger.
Hike on.
Prememost great, come on. Quick.
- I ampeared at Hogwarts
[Nater atte this Harry but nothing buckeded] Of course! Right. Mrs lide girls do you skide sounds] Harry!
Harry: Whoa!! [Harry is left by himself himbin again.

Hey, Neville, Normella. The bitch! [Hagrid then Harry,
that the pointing behind to with histerfacter at him after his eaf gille


In [112]:
print(generate_text(model,"Dumbledore",gen_size=500))

Dumbledore think!
Hagrid: Oh, nixe to meet my dad mid to sevorm to the OMENNESERNEBE
What?
Look was we are to find.
Ron: [glances at the scroom] [Harry notices this.]
Filch: Ravenglaw?
If on e stops, he is ready."
Ron appear, right?
Well, make warno louses.

DUMBLEDORE
Well done! Lee've wanted you too me?
Petunia: It'll be dead by a witchack.
Ron: Well, you got worse the blood that fursthing better, Hermione.
Finntleain, if you heard...Chofoldelt up on a train conicredor...

And in a first time in suppone


In [113]:
print(generate_text(model,"lord voldemort",gen_size=500))

lord voldemort. Malyon for a pircurn.
So he ridn: Even if I amppeaut, with second, the blood touches his fichtairs across thoughts] Good afternoon, Alowort and ratulares one wells their aunds.
The Hand
Who are you ghat was a boggart.
Harry: Well, well. Ifur lest take a would have secret with just and broken bed.
- What are you doing?
[Harry picks up a reasou and ill min the car in drovence both.
You saw you tell me
Whey are the?

Sonamy liked by noncome saver the mestard is, now, Harry. [Harry glances at him,


In [114]:
print(generate_text(model,"voldemort",gen_size=500))

voldemort. The wizard families as he strange whisper activities.
Very good.
We saw you went with me?
Dobby?
Harry: The Ministry of Magic. A caM Gis Breshif! / I
- Do? Harry.
The louss only vision to supposed to get it.

Now, and you can telling where you're going, this
our sumposes later, Hermione stop, butarestoom, imp. Cornelius Fudge] [gurs] I'll be in his right in very, visione sell of sheking firmy swort. He climes back to her with a spider in defeen] Hehe, not a sco, with Harry objects the wand bul
